In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [2]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [4]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 9
Component distribution: {'min': 1, 'p5': 1, 'max': 11, 'p999': 11, 'p99': 11, 'p1': 1, 'p10': 1, 'p90': 11, 'p50': 1, 'p25': 1, 'p75': 3, 'p95': 11, 'mean': 3.111111111111111}


In [5]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             13
nodePropertiesWritten                                                   28
ranLevels                                                                2
didConverge                                                           True
nodeCount                                                               28
communityCount                                                          10
communityDistribution    {'min': 1, 'p5': 1, 'max': 7, 'p999': 7, 'p99'...
modularity                                                        0.692744
modularities                      [0.5850340136054422, 0.6927437641723355]
postProcessingMillis                                                     9
preProcessingMillis                                                      0
computeMillis                                                           29
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [6]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 56}]

In [7]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [8]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,14,1.0,2.0,2.75,3.0,3.87,4
1,1,10,1.0,1.5,4.50,6.1,6.91,7


In [9]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info

[{'communityId': '0-1',
  'nodes': [{'id': '美鑫美利美元利率變動型終身壽險',
    'description': '保險商品內容說明',
    'type': 'Insurance_product'},
   {'id': '台灣人壽',
    'description': '台灣人壽 美鑫美利美元利率變動型終身壽險',
    'type': 'Organization'}],
  'rels': [{'start': '台灣人壽',
    'description': None,
    'type': 'PROVIDES',
    'end': '美鑫美利美元利率變動型終身壽險'}]},
 {'communityId': '0-18',
  'nodes': [{'id': '保單年度', 'description': None, 'type': None},
   {'id': '增值回饋分享金', 'description': '給付及通知', 'type': 'Concept'},
   {'id': '保險費', 'description': None, 'type': 'Concept'},
   {'id': '第六保單年度', 'description': '保單的第六個年度', 'type': 'Concept'}],
  'rels': [{'start': '保單年度',
    'description': None,
    'type': '適用規定',
    'end': '增值回饋分享金'},
   {'start': '增值回饋分享金',
    'description': None,
    'type': '限制於購買增額繳清保險金額辦理',
    'end': '第六保單年度'},
   {'start': '增值回饋分享金', 'description': None, 'type': '抵繳保險費', 'end': '保險費'}]},
 {'communityId': '0-11',
  'nodes': [{'id': '要保人', 'description': None, 'type': 'Person'},
   {'id': '通知',
    'de

In [10]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """請根據提供同社區的資訊包含 nodes 與 relationships,
產生自然語言的摘要 同社區的資訊:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [12]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-1',
 'summary': '台灣人壽是一家提供保險商品的組織，其產品之一是美鑫美利美元利率變動型終身壽險。這款保險產品的內容說明詳述了該商品的特點和優勢。'}

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

summaries = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

Processing communities: 100%|██████████| 14/14 [00:07<00:00,  1.87it/s]


In [14]:
summaries

[{'community': '0-1',
  'summary': '台灣人壽提供一款名為「美鑫美利美元利率變動型終身壽險」的保險商品。這款保險商品的詳細內容可以通過其說明文件了解。'},
 {'community': '0-21',
  'summary': '本公司是一個組織，與宣告利率這一概念存在關聯。本公司透過按利率儲存生息的方式與宣告利率互動。'},
 {'community': '1-5',
  'summary': '在這個社區中，有三個主要元素。首先是一位要保人，其次是一個名為通知的概念，其描述為應將增值回饋分享金的金額以書面或電子郵件方式通知要保人。最後是一份契約。要保人與契約之間存在一個關係，即要保人可以終止契約。此外，通知與要保人之間也存在一個關係，即通知會以某種方式通知要保人。'},
 {'community': '0-12',
  'summary': '「給付」這個概念描述了身故保險金或喪葬費用保險金的給付。該概念與「身故保險金」之間存在一種支付的關係，即「給付」會支付「身故保險金」。'},
 {'community': '1-3',
  'summary': '在民國九十九年二月三日，若未滿15歲的被保險人在此日期之前（不含）訂立了保險契約，其喪葬費用保險金的給付將依照特定方式辦理。'},
 {'community': '1-11',
  'summary': '台灣人壽提供了一款名為美鑫美利美元利率變動型終身壽險的保險商品。該保險商品包含增值回饋分享金，這些分享金可以用於抵繳保險費。增值回饋分享金的適用規定與保單年度相關，並且在第六保單年度限制於購買增額繳清保險金額辦理。'},
 {'community': '0-26',
  'summary': '該社區的資訊包含以下概念和關係：\n1. 概念「累計儲存生息之金額」和「年複利」，以及一個無類型的節點「增額繳清保險金額」。\n2. 「累計儲存生息之金額」通過計算增額來影響「增額繳清保險金額」。\n3. 「年複利」會通過生息來達成「累計儲存生息之金額」。\n\n總結來說，「年複利」的生息作用會導致「累計儲存生息之金額」的增加，而「累計儲存生息之金額」又會影響「增額繳清保險金額」。'},
 {'community': '0-15',
  'summary': '在這個社區中，描述了一個

In [15]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]